# Account - Owner DK PROD

## Database Summary

In [1]:
Import-Module AWS.Tools.Common, AWS.Tools.DynamoDBv2, AWS.Tools.RDS
Import-Module '/Users/conrad.gauntlett/WorkArea/Repos/DBA_MISC/PowerShell/Modules/DevOpsToolkit'

$Account            = 'DevOps-Owner-DK-Prod'
$RDSSummaryColumns  = 'AvailabilityZone', 'DBSnapshotIdentifier', 'SnapshotCreateTime', 'SnapshotType', 'Status'
$Profiles           = Get-AWSCredentials -ListProfileDetail | Where-Object { $_.ProfileName -match $Account }
$Regions            = "eu-central-1" , "eu-west-2" , "eu-west-1", "us-east-1"
$Summary            = Get-DoAWSDBInformation $Profiles $Regions
$Summary | Format-Table -AutoSize

TERM environment variable not set.
Collected Details: Account - Owner-DK-Prod | Region - eu-central-1
Collected Details: Account - Owner-DK-Prod | Region - eu-west-2
Collected Details: Account - Owner-DK-Prod | Region - eu-west-1
Collected Details: Account - Owner-DK-Prod | Region - us-east-1

Account       AccountID    Region    RDSInstances DynamoDBTables No.
-------       ---------    ------    ------------ -------------- ---
Owner-DK-Prod 619122104288 eu-west-2            0              1   1



### DynamoDB Tables

#### List Tables

In [ ]:
$ddbTables = Get-DoDDBTableInformation $Profiles $Regions 
$ddbTables | Format-Table -AutoSize

#### List Backups

In [ ]:
$BackupsList    = Get-DDBBackupList -ProfileName $Profiles.ProfileName -Region eu-west-2
$Columns        = 'TableName', 'BackupName', 'BackupCreationDateTime',    'BackupType' #  'BackupStatus','BackupSizeBytes',
$BackupsList | Sort-Object BackupCreationDateTime -Descending | Select-Object -Property $Columns | Format-Table -AutoSize

#### Get the latest backup for a table

In [ ]:
$Table = 'terraform-locks'
if($BackupsList){

    $Backup = $BackupsList | Where-Object { $_.TableName -eq $Table } | Sort-Object -Property BackupCreationDateTime -Descending | Select-Object -First 1
    $Bk = Get-DDBBackup -BackupArn $Backup.BackupArn -ProfileName $Profiles.ProfileName
    $Bk.BackupDetails

}

#### Create a new backup for the table

In [ ]:
$Table = 'terraform-locks'
$BackupNew = New-DDBBackup -TableName $Table -ProfileName $Profiles.ProfileName -BackupName "$Table-$(get-date -format "yyyy-MM-dd-HH-mm-ss")" -Region eu-west-2
$BackupNew

#### Check new backup details

In [ ]:
$Bk = Get-DDBBackup -BackupArn $BackupNew.BackupArn -ProfileName $Profiles.ProfileName -Region eu-west-2
$Bk.BackupDetails

#### List all backups

In [ ]:
$BackupsList    = Get-DDBBackupList -ProfileName $Profiles.ProfileName -Region eu-west-2
$Columns        = 'TableName', 'BackupName', 'BackupCreationDateTime',    'BackupType' #  'BackupStatus','BackupSizeBytes',
$BackupsList | Sort-Object BackupCreationDateTime -Descending | Select-Object -Property $Columns | Format-Table -AutoSize

#### Get the oldest backup

In [ ]:
$Table = 'terraform-locks'
$BackupOLDEST = $BackupsList | Where-Object { $_.TableName -eq $Table } | Sort-Object -Property BackupCreationDateTime  | Select-Object -First 1
$Bk = Get-DDBBackup -BackupArn $BackupOLDEST.BackupArn -ProfileName $Profiles.ProfileName
$Bk.BackupDetails

#### Delete the oldest database

In [ ]:
$BackupDelete = Remove-DDBBackup -BackupArn $BackupOLDEST.BackupArn -ProfileName $Profiles.ProfileName -Region eu-west-2 -Confirm:$false
$BackupDelete.BackupDetails

#### List all backups

In [ ]:
$BackupsList    = Get-DDBBackupList -ProfileName $Profiles.ProfileName -Region eu-west-2
$Columns        = 'TableName', 'BackupName', 'BackupCreationDateTime',    'BackupType' #  'BackupStatus','BackupSizeBytes',
$BackupsList | Sort-Object BackupCreationDateTime -Descending | Select-Object -Property $Columns | Format-Table -AutoSize